In [70]:
import pandas as pd

# Läs in Excel-filen
xls = pd.ExcelFile("/Users/zamzamyusuf/Documents/github/datavisualization_project_group2/data/page_2/studerande_over_tid.xlsx")

# Visa alla ark
print(xls.sheet_names)


['Tabell 1']


In [71]:
df_raw = xls.parse(xls.sheet_names[0], skiprows=2)
print(df_raw.columns)

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
      dtype='object')


In [72]:
df_raw.columns = ["SUN_kod", "Utbildningsinriktning", "Åldersgrupp"] + list(range(2005, 2025))

In [73]:
print(df.columns)


Index(['SUN_kod', 'Utbildningsinriktning', 'Unnamed: 2', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
      dtype='object')


In [74]:
df = pd.read_excel("/Users/zamzamyusuf/Documents/github/datavisualization_project_group2/data/page_2/studerande_over_tid.xlsx", header=2)  # eller header=0 eller header=2 beroende på hur filen ser ut
print(df.columns)

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
      dtype='object')


In [83]:
df_cleaned = df.rename(columns={
    'Unnamed: 0': 'Kod',
    'Unnamed: 1': 'Utbildningsinriktning',
    'Unnamed: 2': 'Åldersgrupp'
})

In [85]:
df_cleaned[["SUN_kod", "Utbildningsinriktning"]] = df_cleaned[["SUN_kod", "Utbildningsinriktning"]].ffill()

# Smält endast på års-kolumnerna
df_long = pd.melt(
    df_cleaned,
    id_vars=["SUN_kod", "Utbildningsinriktning", "Åldersgrupp"],
    var_name="År",
    value_name="Antal studerande"
)

# Ta bort rader utan värde
df_long = df_long.dropna(subset=["Antal studerande"])

# Konvertera år till int
df_long["År"] = df_long["År"].astype(int)


In [86]:
df_long.head()

,SUN_kod,Utbildningsinriktning,Åldersgrupp,År,Antal studerande
0,totalt,Data/It,-24 år,2005,1281
1,totalt,Data/It,25-29 år,2005,584
2,totalt,Data/It,30-34 år,2005,308
3,totalt,Data/It,35-39 år,2005,198
4,totalt,Data/It,40-44 år,2005,137


In [87]:
# Konvertera "Antal studerande" till numerisk och tvinga bort felaktiga värden
df_long["Antal studerande"] = pd.to_numeric(df_long["Antal studerande"], errors="coerce")

# Rensa bort rader där konverteringen misslyckades
df_long = df_long.dropna(subset=["Antal studerande"])

# Skapa KPI-dataframes för EDA och filtrering
df_total_per_year = df_long.groupby("År")["Antal studerande"].sum().reset_index()
df_total_per_area_year = df_long.groupby(["Utbildningsinriktning", "År"])["Antal studerande"].sum().reset_index()
df_total_per_age_year = df_long.groupby(["Åldersgrupp", "År"])["Antal studerande"].sum().reset_index()
df_total_per_area = df_long.groupby("Utbildningsinriktning")["Antal studerande"].sum().reset_index()

df_long.head(), df_total_per_year.head(), df_total_per_area_year.head(), df_total_per_area.head()

(  SUN_kod Utbildningsinriktning Åldersgrupp    År  Antal studerande
 0  totalt               Data/It      -24 år  2005            1281.0
 1  totalt               Data/It    25-29 år  2005             584.0
 2  totalt               Data/It    30-34 år  2005             308.0
 3  totalt               Data/It    35-39 år  2005             198.0
 4  totalt               Data/It    40-44 år  2005             137.0,
      År  Antal studerande
 0  2005           24787.0
 1  2006           29554.0
 2  2007           33680.0
 3  2008           36625.0
 4  2009           39356.0,
   Utbildningsinriktning    År  Antal studerande
 0               Data/It  2005            2587.0
 1               Data/It  2006            2546.0
 2               Data/It  2007            2797.0
 3               Data/It  2008            2817.0
 4               Data/It  2009            2857.0,
                      Utbildningsinriktning  Antal studerande
 0                                  Data/It          132050.0
 1 

In [96]:
import plotly.express as px

fig = px.line(
    df_long,
    x="År",
    y="Antal studerande",
    color="Utbildningsinriktning",
    title="Antal studerande över tid per utbildningsinriktning",
    labels={"Antal studerande": "Antal studerande", "År": "År"}
)

fig.update_layout(
    xaxis=dict(tickangle=45),
    legend=dict(title="Utbildningsinriktning", orientation="v", x=1.05, y=1)
)

fig.show()


In [161]:
import plotly.express as px

fig = px.bar(
    df_long,
    x="År",
    y="Antal studerande",
    color="Utbildningsinriktning",
    title="Antal studerande per år och utbildningsinriktning",
    labels={"Antal studerande": "Antal studerande", "År": "År"},
    barmode='group'  # Gruppade staplar per år
)

fig.update_layout(
    xaxis=dict(tickangle=45),
    legend=dict(title="Utbildningsinriktning", orientation="v", x=1.05, y=1)
)

fig.show()


In [ ]:
print(df_long.info())
print(df_long.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Kön               1300 non-null   object 
 1   Ålder             420 non-null    object 
 2   År                1300 non-null   int64  
 3   Antal studerande  420 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 40.8+ KB
None
      Kön     Ålder    År  Antal studerande
0  totalt    totalt  2005           24789.0
1  totalt    -24 år  2005            9987.0
2  totalt  25-29 år  2005            5593.0
3  totalt  30-34 år  2005            3289.0
4  totalt  35-39 år  2005            2549.0


In [98]:
df_long.groupby("År")["Antal studerande"].sum().reset_index()

,År,Antal studerande
0,2005,24787.0
1,2006,29554.0
2,2007,33680.0
3,2008,36625.0
4,2009,39356.0
5,2010,41536.0
6,2011,42383.0
7,2012,40579.0
8,2013,42372.0
9,2014,44678.0


In [99]:
df_long.groupby("Utbildningsinriktning")["Antal studerande"].sum().sort_values(ascending=False).head(10)

Utbildningsinriktning
Ekonomi, administration och försäljning         259478.0
Teknik och tillverkning                         154563.0
Hälso- och sjukvård samt socialt arbete         139166.0
Data/It                                         132050.0
Samhällsbyggnad och byggteknik                  131077.0
Kultur, media och design                         62421.0
Hotell, restaurang och turism                    55044.0
Lantbruk, djurvård, trädgård, skog och fiske     31017.0
Transporttjänster                                29945.0
Pedagogik och undervisning                       14935.0
Name: Antal studerande, dtype: float64

In [105]:
df_long[df_long["År"] == 2022]

,SUN_kod,Utbildningsinriktning,Åldersgrupp,År,Antal studerande
2193,totalt,Data/It,-24 år,2022,3163.0
2194,totalt,Data/It,25-29 år,2022,3822.0
2195,totalt,Data/It,30-34 år,2022,3428.0
2196,totalt,Data/It,35-39 år,2022,2014.0
2197,totalt,Data/It,40-44 år,2022,991.0
...,...,...,...,...,...
2272,totalt,Transporttjänster,25-29 år,2022,362.0
2273,totalt,Transporttjänster,30-34 år,2022,271.0
2274,totalt,Transporttjänster,35-39 år,2022,160.0
2275,totalt,Transporttjänster,40-44 år,2022,121.0


In [195]:
import plotly.express as px
import plotly.graph_objects as go

# Fokusområde
highlight_area = "Ekonomi, administration och försäljning"

# Skapa basfiguren
fig = px.bar(
    df_long,
    x="År",
    y="Antal studerande",
    color="Utbildningsinriktning",
    title="Investera i framtiden: Ekonomi och administration växer snabbt bland YH-studenter",
    labels={"Antal studerande": "Antal studerande", "År": "År"},
    barmode='group'
)

# Gör färger statiska med manuell färgsättning (grå för alla, blå för highlight)
utbildningsområden = df_long["Utbildningsinriktning"].unique()
colors = {
    area: "lightgray" if area != highlight_area else "royalblue"
    for area in utbildningsområden
}
fig.for_each_trace(
    lambda trace: trace.update(marker_color=colors.get(trace.name, "lightgray"))
)

# Ta fram sista året och dess värde för pilen
latest_year = df_long["År"].max()
highlight_data = df_long[
    (df_long["Utbildningsinriktning"] == highlight_area) & (df_long["År"] == latest_year)
]

if not highlight_data.empty:
    y_value = highlight_data["Antal studerande"].values[0]
    fig.add_annotation(
        x=latest_year,
        y=y_value,
        text=" Tillväxtmöjlighet",
        showarrow=True,
        arrowhead=3,
        arrowcolor="royalblue",
        ax=0,
        ay=60,
        font=dict(color="royalblue", size=13, family="Arial"),
        bgcolor="white",
        bordercolor="royalblue",
        borderwidth=1
    )

# Anpassa layout för storytelling
fig.update_layout(
    xaxis=dict(tickangle=0),
    legend_title_text="Utbildningsinriktning",
    plot_bgcolor="white",
    paper_bgcolor="white",
    showlegend=True,
    hovermode=False  # Gör grafen mindre interaktiv
)

# Ta bort hovereffekter (gör den statisk)
fig.update_traces(hoverinfo="skip", hovertemplate=None)

# Spara figuren som en större PNG-bild
fig.write_image("assets/figures/education_storytelling.png", width=1920, height=1080, scale=2)

fig.show()


In [192]:
# Ta fram maxvärdet för y-axeln (Antal studerande) i hela df_long för att placera texten ovanför staplarna
max_y = df_long["Antal studerande"].max()

fig.add_annotation(
    x=2010,                # placera texten vid år 2010 på x-axeln
    y=max_y + max_y*0.1,   # placera texten 10% ovanför maxvärdet på y-axeln
    text="🚀 Tillväxtmöjlighet",
    showarrow=True,
    arrowhead=3,
    arrowcolor="royalblue",
    ax=40,                 # pilens startpunkt i pixlar från texten horisontellt (positivt = pil pekar höger)
    ay=0,                  # pilens startpunkt vertikalt (0 = pilen är horisontell)
    font=dict(color="royalblue", size=13, family="Arial"),
    bgcolor="white",
    bordercolor="royalblue",
    borderwidth=1
)


In [178]:
import matplotlib.pyplot as plt

def create_storytelling_chart_matplotlib(df_summary):
    highlight_area = "Ekonomi, administration och försäljning"

    fig, ax = plt.subplots(figsize=(8, 5))

    # Färger: royalblue för highlight, lightgray för andra
    colors = ['royalblue' if area == highlight_area else 'lightgray' for area in df_summary["Utbildningsinriktning"]]

    # Horisontell stapelplot
    ax.barh(df_summary["Utbildningsinriktning"], df_summary["Antal studerande"], color=colors)

    ax.set_xlabel("Antal studerande")
    ax.set_ylabel("Utbildningsområde")
    ax.set_title("Vilket utbildningsområde bör # The Skool fokusera på ?")

    # Ta bort rutnät och axellinjer om du vill (som i Plotly)
    ax.grid(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

    # Lägg till annotation/pil om highlight_area finns
    if highlight_area in df_summary["Utbildningsinriktning"].values:
        highlight_idx = df_summary.index[df_summary["Utbildningsinriktning"] == highlight_area][0]
        highlight_value = df_summary.loc[highlight_idx, "Antal studerande"]

        ax.annotate(
            "Viktig tillväxtmöjlighet",
            xy=(highlight_value, highlight_idx),
            xytext=(highlight_value + 20, highlight_idx),
            va='center',
            ha='left',
            fontsize=12,
            color='royalblue',
            arrowprops=dict(facecolor='royalblue', shrink=0.05, width=1, headwidth=8)
        )

    plt.tight_layout()
    return fig
